# Setting up a mock DB to store Songs, users, and playlists

![](https://github.com/spotify-song/DS/blob/master/api/data/DB_tables_illu.png?raw=true)

In [3]:
import sys
sys.path.insert(0, '../')
from spotify_users import User
user = User()
token_dict = user.user_top_50(user_id='Agustinvargas')

In [4]:
# token_dict['Tokens Info']['access_token']
# token_dict['Tokens Info']['token_type']
# token_dict['Tokens Info']['expires_in']
# token_dict['Tokens Info']['scope']
# token_dict['Tokens Info']['expires_at']
# token_dict['Tokens Info']['refresh_token']
token_dict['User ID']
token_dict['Display Name']

'Agustinvargas'

## Adding user's `display_name` to the users table

In [5]:
from os import getenv
from dotenv import load_dotenv
import psycopg2

import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref

load_dotenv()

# connecting to DB
engine = create_engine(getenv('DATABASE_URL'))
Session = sessionmaker(
                autocommit=False,
                autoflush=False,
                bind=engine
                )

Base = declarative_base()

In [6]:
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True, index=True)
    display_name = Column(String, unique=True, nullable=False)
    
    token = relationship('Tokens', backref='user_token', uselist=False)
    
    def __repr__(self):
        return "<User(display_name='%s')>" % (self.display_name)



class Tokens(Base):
    __tablename__ = 'tokens'
    
    id = Column(Integer, primary_key=True, index=True)
    access_token = Column(String, unique=True, nullable=False)
    token_type = Column(String, unique=False, nullable=False)
    expires_in = Column(Integer, unique=False, nullable=False)
    refresh_token = Column(String, unique=True, nullable=False)
    scope = Column(String, unique=False, nullable=False)
    expires_at = Column(Integer, unique=False, nullable=False)
    user = Column(String, unique=True, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    
    def __repr__(self):
        return "<Tokens(\
        access_token='%s',\
        token_type='%s',\
        expires_in='%s',\
        refresh_token='%s',\
        scope='%s',\
        expires_at='%s',\
        user='%s',\
        user_id='%s')>" % (
            self.id,
            self.access_token,
            self.token_type,
            self.expires_in,
            self.refresh_token,
            self.scope,
            self.expires_at,
            self.user,
            self.user_id)



In [7]:
u1 = User(display_name=token_dict['Display Name'])
u1

<User(display_name='Agustinvargas')>

In [8]:
session = Session()
session.add(u1)
session.commit()

In [9]:
t1 = Tokens(
        access_token=token_dict['Tokens Info']['access_token'],
        token_type=token_dict['Tokens Info']['token_type'],
        expires_in=token_dict['Tokens Info']['expires_in'],
        refresh_token=token_dict['Tokens Info']['refresh_token'],
        scope=token_dict['Tokens Info']['scope'],
        expires_at=token_dict['Tokens Info']['expires_at'],
        user=token_dict['User ID'],
        user_token=u1
        )

In [10]:
session.add(t1)
session.commit()

In [11]:
session.close()